### 6-2-4 メソッドの定義

リスト 6.13 継承とincludeした時のメソッド優先順序

In [1]:
module FirstIncludeModule
  def same_name_method
    :first_include_module
  end
end

module SecondIncludeModule
  def same_name_method
    :second_include_module
  end
end

class Klass
  include FirstIncludeModule
  include SecondIncludeModule
end

object = Klass.new
object.same_name_method

:second_include_module

In [2]:
class Klass
  define_method :same_name_method, FirstIncludeModule.instance_method(:same_name_method)
end

object = Klass.new
object.same_name_method

:first_include_module

### 6-2-5 MethodTransplanting

In [4]:
module UnIncludeModule
  def un_include_module_method
    :un_include_module_method
  end
end

class Klass2
  # TypeErrr にならない
  define_method :un_include_module_method, UnIncludeModule.instance_method(:un_include_module_method)
end

object = Klass2.new
object.un_include_module_method

:un_include_module_method

## 6-3 特異クラス
### 6-3-1 特異クラスと特異メソッド

In [5]:
class OriginalClass
end

obj = OriginalClass.new

def obj.new_singleton_method
  :new_singleton_method
end

:new_singleton_method

In [7]:
obj.class.method_defined? :new_singleton_method

false

In [8]:
obj.singleton_class.method_defined? :new_singleton_method

true

### 6-3-2 特異クラスの確認方法

In [9]:
class Klass3; end

p klass = Klass3.new
klass.singleton_class

#<Klass3:0x00007fd7bfb0a1f0>


#<Class:#<Klass3:0x00007fd7bfb0a1f0>>

In [10]:
KLASS_OBJECT = klass

class << klass; self; end == KLASS_OBJECT.singleton_class

true

### 6-3-3 特異クラスが作成されるタイミング

In [11]:
obj = Object.new

def obj.define_singleton_method
  :singleton_method
end

:define_singleton_method

In [12]:
class << obj
end

In [13]:
obj.singleton_class

#<Class:#<Object:0x00007fd7bf9819a0>>

### 6-3-4 オブジェクトと特異クラス

In [14]:
class OriginalClass2
end

obj = OriginalClass2.new
obj.singleton_class.superclass

OriginalClass2

In [15]:
p obj.instance_of? obj.singleton_class
obj.instance_of? OriginalClass2

false


true

### 6-3-5 特異クラスを持てないオブジェクト

In [16]:
1.singleton_class

TypeError: can't define singleton

In [17]:
:symbol.singleton_class

TypeError: can't define singleton

In [18]:
p true.singleton_class
p false.singleton_class
nil.singleton_class

TrueClass
FalseClass


NilClass

### 6-3-6 特異クラスの特徴

In [19]:
class Klass3; end

p klass = Klass.new

p EigenClass = klass.singleton_class

EigenClass.new

#<Klass:0x00007fd7c0a53fa8>
#<Class:#<Klass:0x00007fd7c0a53fa8>>


TypeError: can't create instance of singleton class

In [20]:
class NewClass < EigenClass
end

TypeError: can't make subclass of singleton class

### 6-3-7 特異クラスとObject#extend

In [21]:
module ExtendedModule
  def extend_method
    :extend_method
  end
end

obj = Object.new
obj.extend ExtendedModule

obj.extend_method

:extend_method

In [23]:
obj = Object.new
p obj.singleton_class.ancestors

obj.extend ExtendedModule
obj.singleton_class.ancestors

[#<Class:#<Object:0x00007fd7bfbd98d8>>, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]


[#<Class:#<Object:0x00007fd7bfbd98d8>>, ExtendedModule, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]

### 6-3-8 特異クラスを使う

In [24]:
SINGLETON_OBJECT = Object.new

#<Object:0x00007fd7bfc025d0>

In [25]:
class << SINGLETON_OBJECT
  def only_method
    :only_method
  end
end

:only_method

In [26]:
SingletonClass = SINGLETON_OBJECT.singleton_class

#<Class:#<Object:0x00007fd7bfc025d0>>

In [27]:
def SingletonClass.instance_get
  SINGLETON_OBJECT
end

:instance_get

In [28]:
SingletonClass.instance_get.only_method

:only_method